In [1]:
import json
import random

In [4]:
!ls CLEVR_CoGenT_v1.0/scenes

CLEVR_trainA_scenes.json  CLEVR_valA_scenes.json  CLEVR_valB_scenes.json


In [15]:
split = "valB"
clevr_train_json = f"CLEVR_CoGenT_v1.0/scenes/CLEVR_{split}_scenes.json"
train_qs = f"CLEVR_CoGenT_v1.0/questions/CLEVR_{split}_questions.json"
data = json.load(open(clevr_train_json))
qs = json.load(open(train_qs))

In [16]:
print(data['scenes'][0]['objects'])
print("len: ", len(data['scenes'][0]['objects']))

[{'pixel_coords': [343, 131, 11.278693199157715], 'size': 'small', 'color': 'green', 'material': 'metal', 'shape': 'sphere', '3d_coords': [0.9906095862388611, 2.083291530609131, 0.3499999940395355], 'rotation': 107.73596690369371}, {'pixel_coords': [396, 172, 9.857704162597656], 'size': 'small', 'color': 'cyan', 'material': 'rubber', 'shape': 'sphere', '3d_coords': [2.69626522064209, 1.5257188081741333, 0.3499999940395355], 'rotation': 305.3536122513589}, {'pixel_coords': [115, 182, 8.91348934173584], 'size': 'large', 'color': 'yellow', 'material': 'rubber', 'shape': 'cylinder', '3d_coords': [0.049163494259119034, -2.864100217819214, 0.699999988079071], 'rotation': 161.8370138842408}, {'pixel_coords': [203, 131, 10.548327445983887], 'size': 'large', 'color': 'purple', 'material': 'rubber', 'shape': 'cube', '3d_coords': [-0.4719269275665283, -0.5699371695518494, 0.699999988079071], 'rotation': 159.41862667811446}, {'pixel_coords': [253, 75, 13.141877174377441], 'size': 'large', 'color':

In [17]:
def object_info_to_description(object_list):
    descriptions = []
    random.shuffle(object_list)
    for obj in object_list:
        desc = f"A {obj['size']} {obj['color']} {obj['material']} {obj['shape']}"
        desc += f" rotated {obj['rotation']:.1f}° located at"
        desc += f" 3D coordinates ({obj['3d_coords'][0]:.2f}, {obj['3d_coords'][1]:.2f}, {obj['3d_coords'][2]:.2f})"
        desc += f" and pixel coordinates ({obj['pixel_coords'][0]}, {obj['pixel_coords'][1]}, {obj['pixel_coords'][2]:.2f})"
        descriptions.append(desc)
    
    final_description = "Scene Description:\n"
    for i, desc in enumerate(descriptions, 1):
        final_description += f"{desc}\n"
    
    return final_description

In [18]:
object_info_to_description(data['scenes'][0]['objects'])

'Scene Description:\nA large yellow rubber cylinder rotated 161.8° located at 3D coordinates (0.05, -2.86, 0.70) and pixel coordinates (115, 182, 8.91)\nA large purple rubber cube rotated 159.4° located at 3D coordinates (-0.47, -0.57, 0.70) and pixel coordinates (203, 131, 10.55)\nA large red rubber cube rotated 37.4° located at 3D coordinates (-2.04, 2.22, 0.70) and pixel coordinates (253, 75, 13.14)\nA small green metal sphere rotated 107.7° located at 3D coordinates (0.99, 2.08, 0.35) and pixel coordinates (343, 131, 11.28)\nA small cyan rubber sphere rotated 305.4° located at 3D coordinates (2.70, 1.53, 0.35) and pixel coordinates (396, 172, 9.86)\n'

In [19]:
img2obj_dict = {}
for scene in data['scenes']:
    obj_list = scene['objects']
    img2obj_dict[scene['image_filename']] = obj_list

In [20]:
qs['questions'][0]

{'question_index': 0,
 'question_family_index': 29,
 'image_index': 0,
 'question': 'The big thing that is in front of the large rubber cube in front of the small thing that is behind the tiny matte ball is what color?',
 'answer': 'yellow',
 'image_filename': 'CLEVR_valB_000000.png',
 'split': 'valB',
 'program': [{'value_inputs': [], 'inputs': [], 'function': 'scene'},
  {'value_inputs': ['small'], 'inputs': [0], 'function': 'filter_size'},
  {'value_inputs': ['rubber'], 'inputs': [1], 'function': 'filter_material'},
  {'value_inputs': ['sphere'], 'inputs': [2], 'function': 'filter_shape'},
  {'value_inputs': [], 'inputs': [3], 'function': 'unique'},
  {'value_inputs': ['behind'], 'inputs': [4], 'function': 'relate'},
  {'value_inputs': ['small'], 'inputs': [5], 'function': 'filter_size'},
  {'value_inputs': [], 'inputs': [6], 'function': 'unique'},
  {'value_inputs': ['front'], 'inputs': [7], 'function': 'relate'},
  {'value_inputs': ['large'], 'inputs': [8], 'function': 'filter_siz

In [21]:
random.shuffle(qs['questions'])
cnt = 0 
qa_pairs = [] 
added_pair = set()
for qd in qs['questions']:
    img_idx = qd['image_filename']
    total_count = len(img2obj_dict[img_idx]) # object list length
    desc = object_info_to_description(img2obj_dict[img_idx])
    question, answer = qd['question'], qd['answer']
    if 'how many' in question.lower() or 'number' in question.lower():
        qa_pairs.append({
            "img_filename": img_idx,
            'q': question,
            'a': answer,
            'description': desc 
        })
        if img_idx not in added_pair:
            qa_pairs.append({
                "img_filename": img_idx,
                'q': "How many items are there in the described scene?",
                'a': total_count,
                'description': desc 
            })
            added_pair.add(img_idx)


In [22]:
len(qa_pairs)

59978

In [23]:
len(added_pair)

14884

In [24]:
# save for later loading
with open(f"clever_counting_problems_clevr_cogent_v1.0_{split}.json", 'w') as fw:
    json.dump( qa_pairs, fw, indent=4)

In [20]:
random.shuffle(qa_pairs)

In [57]:
qa_pairs[:2]

[{'img_filename': 'CLEVR_trainA_048403.png',
  'q': 'How many things are both on the right side of the big yellow rubber thing and left of the purple ball?',
  'a': '5',
  'description': 'Scene Description:\nA large red rubber cylinder rotated 291.3° located at 3D coordinates (-0.89, -2.73, 0.70) and pixel coordinates (101, 152, 10.04)\nA small purple metal sphere rotated 247.7° located at 3D coordinates (2.93, 0.87, 0.35) and pixel coordinates (379, 183, 9.66)\nA large cyan rubber cylinder rotated 114.5° located at 3D coordinates (-2.40, 2.23, 0.70) and pixel coordinates (246, 82, 13.94)\nA small red metal cylinder rotated 109.9° located at 3D coordinates (-0.95, 1.77, 0.35) and pixel coordinates (270, 113, 12.83)\nA small red rubber cylinder rotated 343.7° located at 3D coordinates (-0.12, -0.74, 0.35) and pixel coordinates (209, 153, 10.82)\nA large red rubber cylinder rotated 324.5° located at 3D coordinates (-2.71, -2.21, 0.70) and pixel coordinates (84, 119, 11.59)\nA small red m

In [26]:
def query_r1(query='who are you?', model="deepseek-ai/DeepSeek-R1"):
    # Create the chat completion
    response = client.chat.completions.create(
        model=model,
         messages=[
            {'role': 'user', 
            'content': query}
        ],
        stream=False,
    )
    # Print the response
    return response.choices[0].message.content

In [44]:
def format_query(qa_dict):
    query = "Answer the question according to scene description.\n\n"
    query += qa_dict['description']
    query += f"\nQuestion:\n{qa_dict['q']}"
    return query 
    

In [39]:
debug_query = format_query(qa_pairs[0])
print(query_r1(debug_query))

<think>Okay, let's see. The user is asking how many items are there in the described scene. Let me go through the scene description step by step.

So, the scene description lists each object with details like color, material, shape, rotation, 3D coordinates, and pixel coordinates. Each entry starts with "A" which usually indicates one item each. Let me count each one.

First entry: "A small green metal cylinder..." That's one. Second: "A small blue rubber cylinder..." Second item. Third: "A small cyan rubber cylinder..." That's three. Fourth: "A large cyan metal sphere..." Four. Fifth: "A large brown metal cube..." Five. Sixth: "A large yellow rubber cube..." Six. Seventh: "A large brown rubber cylinder..." That's seven. 

Wait, did I miss any? Let me check again. The list has entries from "A small green..." up to the seventh one. Each sentence starts with "A", which suggests each is a separate item. No commas separating multiple items in a single entry. Each has different attributes a

In [41]:
print(debug_query)

Answer the question accordingly to scene description.

Scene Description:
A small green metal cylinder rotated 329.5° located at 3D coordinates (-2.49, -1.65, 0.35) and pixel coordinates (111, 132, 11.81)
A small blue rubber cylinder rotated 312.2° located at 3D coordinates (-1.73, -2.91, 0.35) and pixel coordinates (76, 163, 10.57)
A small cyan rubber cylinder rotated 48.4° located at 3D coordinates (-2.10, -0.22, 0.35) and pixel coordinates (172, 118, 12.41)
A large cyan metal sphere rotated 27.4° located at 3D coordinates (1.52, -1.26, 0.70) and pixel coordinates (247, 181, 9.33)
A large brown metal cube rotated 107.7° located at 3D coordinates (-0.73, 2.39, 0.70) and pixel coordinates (290, 92, 12.93)
A large yellow rubber cube rotated 288.2° located at 3D coordinates (0.52, 0.63, 0.70) and pixel coordinates (279, 130, 11.09)
A large brown rubber cylinder rotated 229.8° located at 3D coordinates (2.38, 0.38, 0.70) and pixel coordinates (343, 166, 9.77)

Question:
How many items are

In [42]:
qa_pairs[1]

{'img_filename': 'CLEVR_train_044000.png',
 'q': 'How many rubber objects are either small blue spheres or small things?',
 'a': '2',
 'description': 'Scene Description:\nA large purple rubber sphere rotated 78.4° located at 3D coordinates (2.27, 0.87, 0.70) and pixel coordinates (360, 156, 9.49)\nA large gray metal cube rotated 152.7° located at 3D coordinates (2.79, -1.26, 0.70) and pixel coordinates (301, 213, 7.91)\nA large purple metal sphere rotated 79.2° located at 3D coordinates (-2.66, -2.74, 0.70) and pixel coordinates (51, 126, 10.61)\nA large blue rubber sphere rotated 279.5° located at 3D coordinates (1.31, 2.72, 0.70) and pixel coordinates (376, 112, 11.19)\nA small brown rubber cube rotated 124.1° located at 3D coordinates (-2.49, 2.61, 0.35) and pixel coordinates (251, 82, 13.79)\nA small green rubber sphere rotated 323.9° located at 3D coordinates (-2.02, 0.45, 0.35) and pixel coordinates (197, 109, 12.22)\n'}

In [45]:
debug_query1 = format_query(qa_pairs[1])
res1 = query_r1(debug_query1)

<think>Okay, let's tackle this question. The user wants to know how many rubber objects are either small blue spheres or small things. Hmm, first, I need to parse each part of the question correctly.

Let's go back to the scene description. The objects are listed with their attributes: size, color, material, shape, rotation, 3D and pixel coordinates. The key here is to filter the objects based on the criteria given.

The question has two parts: "small blue spheres" OR "small things". So any rubber object that is either a small blue sphere or any small thing (regardless of other attributes) counts. But wait, do both categories need to be rubber? Because the question says "rubber objects are either...". So rubber is the material, and the condition is either being a small blue sphere or a small thing. So first, all rubber objects, and among them, count those that are either small blue spheres or small (regardless of color or shape). Wait, no. Let me re-read the question.

The question is:

In [47]:
qa_pairs[1]

{'img_filename': 'CLEVR_train_044000.png',
 'q': 'How many rubber objects are either small blue spheres or small things?',
 'a': '2',
 'description': 'Scene Description:\nA large purple rubber sphere rotated 78.4° located at 3D coordinates (2.27, 0.87, 0.70) and pixel coordinates (360, 156, 9.49)\nA large gray metal cube rotated 152.7° located at 3D coordinates (2.79, -1.26, 0.70) and pixel coordinates (301, 213, 7.91)\nA large purple metal sphere rotated 79.2° located at 3D coordinates (-2.66, -2.74, 0.70) and pixel coordinates (51, 126, 10.61)\nA large blue rubber sphere rotated 279.5° located at 3D coordinates (1.31, 2.72, 0.70) and pixel coordinates (376, 112, 11.19)\nA small brown rubber cube rotated 124.1° located at 3D coordinates (-2.49, 2.61, 0.35) and pixel coordinates (251, 82, 13.79)\nA small green rubber sphere rotated 323.9° located at 3D coordinates (-2.02, 0.45, 0.35) and pixel coordinates (197, 109, 12.22)\n'}